source: https://github.com/minoguep/rhyme_detection and https://paulminogue.com/index.php/2021/02/14/using-a-siamese-neural-network-to-create-a-simple-rhyme-detector/

In [46]:
import string
import json

import nltk
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Subtract
from tensorflow.keras.callbacks import ModelCheckpoint, TerminateOnNaN, CSVLogger, EarlyStopping

tqdm.pandas()

MAX_LEN = 64
SEED = 420
# sets random, np.random and tf.random seed
tf.keras.utils.set_random_seed(
    SEED
)

### Step 1: Create dataset
We want equally many positive and negative samples of rhyme pairs

In [47]:
pos = pd.read_csv("positive_pairs.tsv", sep="\t")
full_neg = pd.read_csv("negative_pairs.tsv", sep="\t")
neg = full_neg.sample(n=len(pos), random_state=SEED)
df = pd.concat([pos, neg])
df = df.reset_index()
df

,index,word_a,word_b,rhyme
0,0,stall,skrall,1
1,1,ving,ting,1
2,2,orden,horden,1
3,3,taler,svaler,1
4,4,juleskikk,blikk,1
...,...,...,...,...
14471,637,hatt,slott,0
14472,13030,mur,stund,0
14473,1029,hud,mave,0
14474,16689,dør,sans,0


In [48]:
all_data = pd.read_csv("tita_rhymes_poems.tsv", sep="\t")

all_text = ""
for e in all_data.stanza:
    all_text += e

### Step 2: Create model
Copy paste from Pauls notebook

In [49]:
def tokenize_inputs(phrase_a, phrase_b, tokenizer):
    tokenized_phrases = tokenizer.texts_to_sequences([phrase_a, phrase_b])

    # now loop through inputs and pad or reduce size if required
    tokenized_phrases_for_output = []
    for phrase in tokenized_phrases:
        if len(phrase) < MAX_LEN:
            length_to_pad = MAX_LEN - len(phrase)
            phrase_for_output = ([0] * length_to_pad) + phrase
        elif len(phrase) > MAX_LEN:
            phrase_for_output = phrase[-MAX_LEN:]
        else:
            phrase_for_output = phrase
        tokenized_phrases_for_output.append(phrase_for_output)

    return tf.constant(tokenized_phrases_for_output, dtype=tf.float64)

In [50]:
tokenizer = Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(all_text)

df['word_tokens'] = df.progress_apply(
    lambda row: tokenize_inputs(row['word_a'], row['word_b'], tokenizer), axis=1
)

tokenizer_config = tokenizer.to_json()

with open('tokenizer_config.json', 'w') as f:
    f.write(tokenizer_config)

  0%|          | 0/14476 [00:00<?, ?it/s]

In [51]:
def create_model():
  
    word_a_input_tokens = Input(
      shape=(MAX_LEN, 1), name='word_a_input_tokens'
      )
    word_b_input_tokens = Input(
      shape=(MAX_LEN, 1), name='word_b_input_tokens'
      )
    
    # This is the siamese portion of the model 
    common_lstm = LSTM(64, return_sequences=False, activation="relu", name="common_lstm_layer")

    word_a_lstm_output = common_lstm(word_a_input_tokens)
    word_b_lstm_output = common_lstm(word_b_input_tokens)

    #concatenate_lstm_outputs
    concat_layer = Subtract(name="concatenate_lstm_outputs")(
      [word_a_lstm_output, word_b_lstm_output]
      )
    
    # dense layers before final classification
    dense_layers = Dense(64, activation="relu", name="first_dense_layer")(concat_layer)
    dense_layers = Dropout(0.5)(dense_layers)

    dense_layers = Dense(32, activation="relu", name="second_dense_layer")(dense_layers)
    dense_layers = Dropout(0.5)(dense_layers)

    dense_layers = Dense(8, activation="relu", name="third_dense_layer")(dense_layers)
    dense_layers = Dropout(0.5)(dense_layers)

    classification_layer = Dense(1, activation="sigmoid", name="classification_layer")(dense_layers)
    
    model = Model(
      inputs=[word_a_input_tokens, word_b_input_tokens], 
      outputs = classification_layer
      )

    model.compile(
      loss="binary_crossentropy",
      metrics=["accuracy"],
      optimizer="Adam"
    )

    return model

In [52]:
X_train_indexes, X_test_indexes, y_train, y_test = train_test_split(
    list(df.index), list(df['rhyme']), stratify=df['rhyme'], 
    test_size=0.4, random_state=SEED
    )

X_test_indexes, X_val_indexes, y_test, y_val = train_test_split(
    X_test_indexes, y_test, stratify=y_test, 
    test_size=0.25, random_state=SEED
    )

X_train = tf.convert_to_tensor(list(df.loc[X_train_indexes]["word_tokens"]))
X_val = tf.convert_to_tensor(list(df.loc[X_val_indexes]["word_tokens"]))
X_test = tf.convert_to_tensor(list(df.loc[X_test_indexes]["word_tokens"]))

y_train = tf.convert_to_tensor(y_train)
y_val = tf.convert_to_tensor(y_val)
y_test = tf.convert_to_tensor(y_test)

In [53]:
print(f"""
    Data set size: Full set: {len(df)}
    Train: {len(X_train)}
    Validation: {len(X_val)}
    Test: {len(X_test)}
""")



    Data set size: Full set: 14476
    Train: 8685
    Validation: 1448
    Test: 4343



In [54]:
model_name = "rhyme_model1"

In [55]:
# #Uncomment to train model 

# model = create_model()

# model_checkpoint = ModelCheckpoint(f"models/{model_name}.hdf5",monitor="val_loss")
# terminate_on_nan = TerminateOnNaN()
# csv_logger = CSVLogger(f'logs/training_{model_name}.log')
# early_stop = EarlyStopping(monitor='loss', patience=5)

# history = model.fit(
#     [X_train[:, 0], X_train[:, 1]],
#     y_train,
#     batch_size=128,
#     epochs=100,
#     callbacks=[model_checkpoint, terminate_on_nan, csv_logger, early_stop],
#     validation_data=([X_val[:, 0], X_val[:, 1]], y_val)
# )

In [56]:
# load the model
model = load_model(f"models/{model_name}.hdf5")

y_pred = model.predict([X_test[:, 0], X_test[:, 1]])
y_pred = y_pred > 0.5
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95      2171
           1       0.97      0.92      0.95      2172

    accuracy                           0.95      4343
   macro avg       0.95      0.95      0.95      4343
weighted avg       0.95      0.95      0.95      4343



In [57]:
samples = [
    ["Kan du ikke se det", "Deg skal jeg lede"], 
    ["Kaker av alle slag", "Her henger Norges flagg"], 
    ["Jeg har ikke tid", "Til dette svineri"],
    ["Hva har du sagt", "Kaken er bakt"], 
    ["Barna er lagt", "Kaken er laget"],
    ["Gjorde du det med vilje", "Kaken smaker vanilje"], 
    ["Dette vokser", "Satans underbukser"],
]

sample_tokens = [tokenize_inputs(lyrics[0], lyrics[1], tokenizer) for lyrics in samples]
sample_tokens = tf.convert_to_tensor(sample_tokens)

print(sample_tokens[:, 0].shape, sample_tokens[:, 1].shape)

sample_pred = model.predict([sample_tokens[:, 0], sample_tokens[:, 1]])
predictions = [round(pred[0], 4) for pred in sample_pred]
for i in range(len(samples)):
    print(f"Sentence 1: {samples[i][0]}")
    print(f"Sentence 2: {samples[i][1]}")
    print(f"{'Rhyme' if predictions[i] > 0.5 else 'Non-rhyme'}({predictions[i]})")
    print("---------------\n")

(7, 64) (7, 64)
Sentence 1: Kan du ikke se det
Sentence 2: Deg skal jeg lede
Non-rhyme(0.29179999232292175)
---------------

Sentence 1: Kaker av alle slag
Sentence 2: Her henger Norges flagg
Rhyme(0.9817000031471252)
---------------

Sentence 1: Jeg har ikke tid
Sentence 2: Til dette svineri
Non-rhyme(0.4284999966621399)
---------------

Sentence 1: Hva har du sagt
Sentence 2: Kaken er bakt
Rhyme(0.9890000224113464)
---------------

Sentence 1: Barna er lagt
Sentence 2: Kaken er laget
Non-rhyme(0.0006000000284984708)
---------------

Sentence 1: Gjorde du det med vilje
Sentence 2: Kaken smaker vanilje
Rhyme(0.998199999332428)
---------------

Sentence 1: Dette vokser
Sentence 2: Satans underbukser
Rhyme(0.998199999332428)
---------------



## Try again with mirrored examples included

In [58]:
def get_mirrored_df(df):
    mirror = pd.DataFrame({"word_a": df["word_b"], 
                           "word_b": df["word_a"], 
                           "rhyme": df["rhyme"], 
                           "word_tokens":[(t[1], t[0]) for t in df["word_tokens"]]})
    return pd.concat((df, mirror))

In [59]:
train_df = df.loc[X_train_indexes]
dev_df = df.loc[X_val_indexes]
len(train_df), len(dev_df)

(8685, 1448)

In [60]:
double_train = get_mirrored_df(train_df)
double_train

,index,word_a,word_b,rhyme,word_tokens
13973,18365.0,verdien,fører,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
9981,4141.0,slik,døden,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
4746,4746.0,lanser,stimulanser,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
3150,3150.0,smerte-stønn,bønn,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
12056,13514.0,velte,briste,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
...,...,...,...,...,...
4851,NaN,dynker,rynker,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
4356,NaN,hei,ei,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
12693,NaN,sloet,strand,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
4879,NaN,hud,brud,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."


In [61]:
double_dev = get_mirrored_df(dev_df)
double_dev

,index,word_a,word_b,rhyme,word_tokens
3594,3594.0,beskytte,hytte,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
2736,2736.0,malm,halm,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
14351,19166.0,visen,ganger,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
5492,5492.0,li,forbi,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
9140,1346.0,ene,borg,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
...,...,...,...,...,...
13766,NaN,landet,en,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
9926,NaN,morgen,hjul,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
1196,NaN,ferdselsårer,kårer,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
6586,NaN,finne,noensinne,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."


In [62]:
X_train = tf.convert_to_tensor(list(double_train["word_tokens"]))
X_val = tf.convert_to_tensor(list(double_dev["word_tokens"]))

y_train = tf.convert_to_tensor(list(double_train["rhyme"]))
y_val = tf.convert_to_tensor(list(double_dev["rhyme"]))

In [63]:
model_name = "rhyme_model_mirror"

In [65]:
# # Uncomment to train model 
# model = create_model()

# model_checkpoint = ModelCheckpoint(f"models/{model_name}.hdf5",monitor="val_loss")
# terminate_on_nan = TerminateOnNaN()
# csv_logger = CSVLogger(f'logs/training_{model_name}.log')
# early_stop = EarlyStopping(monitor='loss', patience=5)

# history = model.fit(
#     [X_train[:, 0], X_train[:, 1]],
#     y_train,
#     batch_size=128,
#     epochs=100,
#     callbacks=[model_checkpoint, terminate_on_nan, csv_logger, early_stop],
#     validation_data=([X_val[:, 0], X_val[:, 1]], y_val)
# )

Epoch 1/100
136/136 [==============================] - 8s 44ms/step - loss: 0.6717 - accuracy: 0.5553 - val_loss: 0.5675 - val_accuracy: 0.8622
Epoch 2/100
136/136 [==============================] - 5s 40ms/step - loss: 0.5250 - accuracy: 0.7875 - val_loss: 0.3444 - val_accuracy: 0.8854
Epoch 3/100
136/136 [==============================] - 5s 40ms/step - loss: 0.4022 - accuracy: 0.8272 - val_loss: 0.2755 - val_accuracy: 0.9088
Epoch 4/100
136/136 [==============================] - 6s 42ms/step - loss: 0.3634 - accuracy: 0.8399 - val_loss: 0.2590 - val_accuracy: 0.9157
Epoch 5/100
136/136 [==============================] - 6s 41ms/step - loss: 0.3433 - accuracy: 0.8440 - val_loss: 0.2539 - val_accuracy: 0.9178
Epoch 6/100
136/136 [==============================] - 6s 41ms/step - loss: 0.3298 - accuracy: 0.8500 - val_loss: 0.2478 - val_accuracy: 0.9147
Epoch 7/100
136/136 [==============================] - 6s 43ms/step - loss: 0.3147 - accuracy: 0.8936 - val_loss: 0.2485 - val_accuracy:

In [66]:
# load the model
model = load_model(f"models/{model_name}.hdf5")

y_pred = model.predict([X_test[:, 0], X_test[:, 1]])
y_pred = y_pred > 0.5
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      2171
           1       0.97      0.91      0.94      2172

    accuracy                           0.94      4343
   macro avg       0.94      0.94      0.94      4343
weighted avg       0.94      0.94      0.94      4343



In [67]:
samples = [
    ["Kan du ikke se det", "Deg skal jeg lede"], 
    ["Kaker av alle slag", "Her henger Norges flagg"], 
    ["Jeg har ikke tid", "Til dette svineri"],
    ["Hva har du sagt", "Kaken er bakt"], 
    ["Barna er lagt", "Kaken er laget"],
    ["Gjorde du det med vilje", "Kaken smaker vanilje"], 
    ["Dette vokser", "Satans underbukser"],
]

sample_tokens = [tokenize_inputs(lyrics[0], lyrics[1], tokenizer) for lyrics in samples]
sample_tokens = tf.convert_to_tensor(sample_tokens)

print(sample_tokens[:, 0].shape, sample_tokens[:, 1].shape)

sample_pred = model.predict([sample_tokens[:, 0], sample_tokens[:, 1]])
predictions = [round(pred[0], 4) for pred in sample_pred]
for i in range(len(samples)):
    print(f"Sentence 1: {samples[i][0]}")
    print(f"Sentence 2: {samples[i][1]}")
    print(f"{'Rhyme' if predictions[i] > 0.5 else 'Non-rhyme'}({predictions[i]})")
    print("---------------\n")

(7, 64) (7, 64)
Sentence 1: Kan du ikke se det
Sentence 2: Deg skal jeg lede
Non-rhyme(0.16769999265670776)
---------------

Sentence 1: Kaker av alle slag
Sentence 2: Her henger Norges flagg
Rhyme(0.9994999766349792)
---------------

Sentence 1: Jeg har ikke tid
Sentence 2: Til dette svineri
Non-rhyme(0.4575999975204468)
---------------

Sentence 1: Hva har du sagt
Sentence 2: Kaken er bakt
Rhyme(0.9977999925613403)
---------------

Sentence 1: Barna er lagt
Sentence 2: Kaken er laget
Non-rhyme(0.009700000286102295)
---------------

Sentence 1: Gjorde du det med vilje
Sentence 2: Kaken smaker vanilje
Rhyme(0.9998000264167786)
---------------

Sentence 1: Dette vokser
Sentence 2: Satans underbukser
Rhyme(0.9998000264167786)
---------------



# Try again with different 2:3 positive to negative ratio
Use same test and dev sets, and same tokenizer. Only expand training set.  
Train set is already 50/50 positive and negative. We want to make it 40/60 --> increase negative examples by half of what we already have

In [68]:
new_neg = len(neg) // 2
new_neg

3619

In [69]:
# use same seed, and extract all the pairs already used + the ones we need 
negative2 = full_neg.sample(n=len(pos) + new_neg, random_state=SEED)

In [70]:
unused_neg = negative2[len(pos):]
unused_neg

,word_a,word_b,rhyme
9448,hår,blundet,0
15651,brenner,skatt,0
16533,siv,stille,0
16219,verdensdommen,kronet,0
13700,jord,sans,0
...,...,...,...
5654,sine,garn,0
12379,fløyt,topp,0
5668,forenes,ett,0
11137,nypegren,vår,0


In [71]:
# use same tokenizer as above
unused_neg['word_tokens'] = unused_neg.progress_apply(
    lambda row: tokenize_inputs(row['word_a'], row['word_b'], tokenizer), axis=1
)

unused_neg = get_mirrored_df(unused_neg)

X_train_new = tf.convert_to_tensor(list(X_train) + list(unused_neg["word_tokens"]))
y_train_new = tf.convert_to_tensor(list(y_train) + list(unused_neg["rhyme"]))


  0%|          | 0/3619 [00:00<?, ?it/s]

/tmp/ipykernel_146358/2379011568.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unused_neg['word_tokens'] = unused_neg.progress_apply(


In [72]:
X_train_new.shape

TensorShape([24608, 2, 64])

In [73]:
model_name = "rhyme_model_2_3_ratio"

In [76]:
# # Uncomment to train model
# model = create_model()

# model_checkpoint = ModelCheckpoint(f"models/{model_name}.hdf5",monitor="val_loss")
# terminate_on_nan = TerminateOnNaN()
# csv_logger = CSVLogger(f'logs/training_{model_name}.log')
# early_stop = EarlyStopping(monitor='loss', patience=5)

# history = model.fit(
#     [X_train_new[:, 0], X_train_new[:, 1]],
#     y_train_new,
#     batch_size=128,
#     epochs=100,
#     callbacks=[model_checkpoint, terminate_on_nan, csv_logger, early_stop],
#     validation_data=([X_val[:, 0], X_val[:, 1]], y_val)
# )

Epoch 1/100
193/193 [==============================] - 15s 64ms/step - loss: 0.5300 - accuracy: 0.6943 - val_loss: 0.5166 - val_accuracy: 0.8546
Epoch 2/100
193/193 [==============================] - 12s 60ms/step - loss: 0.4442 - accuracy: 0.8624 - val_loss: 0.4098 - val_accuracy: 0.9037
Epoch 3/100
193/193 [==============================] - 14s 71ms/step - loss: 0.3762 - accuracy: 0.8787 - val_loss: 0.3426 - val_accuracy: 0.9071
Epoch 4/100
193/193 [==============================] - 13s 67ms/step - loss: 0.3361 - accuracy: 0.8882 - val_loss: 0.2759 - val_accuracy: 0.9102
Epoch 5/100
193/193 [==============================] - 11s 59ms/step - loss: 0.3067 - accuracy: 0.8980 - val_loss: 0.2470 - val_accuracy: 0.9088
Epoch 6/100
193/193 [==============================] - 12s 60ms/step - loss: 0.2935 - accuracy: 0.9000 - val_loss: 0.2307 - val_accuracy: 0.9168
Epoch 7/100
193/193 [==============================] - 14s 73ms/step - loss: 0.2807 - accuracy: 0.9043 - val_loss: 0.2183 - val_ac

193/193 [==============================] - 13s 66ms/step - loss: 0.1659 - accuracy: 0.9494 - val_loss: 0.1511 - val_accuracy: 0.9582
Epoch 58/100
193/193 [==============================] - 13s 65ms/step - loss: 0.1629 - accuracy: 0.9506 - val_loss: 0.1708 - val_accuracy: 0.9510
Epoch 59/100
193/193 [==============================] - 14s 72ms/step - loss: 0.1621 - accuracy: 0.9506 - val_loss: 0.1718 - val_accuracy: 0.9555
Epoch 60/100
193/193 [==============================] - 14s 72ms/step - loss: 0.1593 - accuracy: 0.9527 - val_loss: 0.1838 - val_accuracy: 0.9506
Epoch 61/100
193/193 [==============================] - 14s 74ms/step - loss: 0.1589 - accuracy: 0.9517 - val_loss: 0.1702 - val_accuracy: 0.9572
Epoch 62/100
193/193 [==============================] - 13s 67ms/step - loss: 0.1584 - accuracy: 0.9522 - val_loss: 0.1830 - val_accuracy: 0.9520
Epoch 63/100
193/193 [==============================] - 12s 63ms/step - loss: 0.1550 - accuracy: 0.9524 - val_loss: 0.1948 - val_accuracy

In [77]:
# load the model
model = load_model(f"models/{model_name}.hdf5")

y_pred = model.predict([X_test[:, 0], X_test[:, 1]])
y_pred = y_pred > 0.5
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.96      2171
           1       0.97      0.94      0.96      2172

    accuracy                           0.96      4343
   macro avg       0.96      0.96      0.96      4343
weighted avg       0.96      0.96      0.96      4343



In [78]:
samples = [
    ["Kan du ikke se det", "Deg skal jeg lede"], 
    ["Kaker av alle slag", "Her henger Norges flagg"], 
    ["Jeg har ikke tid", "Til dette svineri"],
    ["Hva har du sagt", "Kaken er bakt"], 
    ["Barna er lagt", "Kaken er laget"],
    ["Gjorde du det med vilje", "Kaken smaker vanilje"], 
    ["Dette vokser", "Satans underbukser"],
]

sample_tokens = [tokenize_inputs(lyrics[0], lyrics[1], tokenizer) for lyrics in samples]
sample_tokens = tf.convert_to_tensor(sample_tokens)
sample_pred = model.predict([sample_tokens[:, 0], sample_tokens[:, 1]])
predictions = [round(pred[0], 4) for pred in sample_pred]
for i in range(len(samples)):
    print(f"Lyric 1: {samples[i][0]}")
    print(f"Lyric 2: {samples[i][1]}")
    print(f"{'Rhyme' if predictions[i] > 0.5 else 'Non-rhyme'}({predictions[i]})")
    print("---------------\n")

Lyric 1: Kan du ikke se det
Lyric 2: Deg skal jeg lede
Non-rhyme(0.46810001134872437)
---------------

Lyric 1: Kaker av alle slag
Lyric 2: Her henger Norges flagg
Rhyme(0.9807000160217285)
---------------

Lyric 1: Jeg har ikke tid
Lyric 2: Til dette svineri
Rhyme(0.941100001335144)
---------------

Lyric 1: Hva har du sagt
Lyric 2: Kaken er bakt
Rhyme(0.9742000102996826)
---------------

Lyric 1: Barna er lagt
Lyric 2: Kaken er laget
Non-rhyme(0.0)
---------------

Lyric 1: Gjorde du det med vilje
Lyric 2: Kaken smaker vanilje
Rhyme(0.983299970626831)
---------------

Lyric 1: Dette vokser
Lyric 2: Satans underbukser
Rhyme(0.983299970626831)
---------------



## Use all rhyme pairs

In [ ]:
# unused_neg_2 = neg.sample(n=len(neg), random_state=SEED)[len(negative2):]

# # use same tokenizer as above
# unused_neg_2['word_tokens'] = unused_neg_2.progress_apply(
#     lambda row: tokenize_inputs(row['word_a'], row['word_b'], tokenizer), axis=1
# )

# unused_neg_2 = get_mirrored_df(unused_neg_2)

# X_train_new = tf.convert_to_tensor(list(X_train_new) + list(unused_neg_2["word_tokens"]))
# y_train_new = tf.convert_to_tensor(list(y_train_new) + list(unused_neg_2["rhyme"]))
# X_train_new.shape

# model_name = "rhyme_model_all_pairs"

# # # Uncomment to train model
# # model = create_model()

# # model_checkpoint = ModelCheckpoint(f"models/{model_name}.hdf5",monitor="val_loss")
# # terminate_on_nan = TerminateOnNaN()
# # csv_logger = CSVLogger(f'logs/training_{model_name}.log')
# # early_stop = EarlyStopping(monitor='loss', patience=5)

# # history = model.fit(
# #     [X_train_new[:, 0], X_train_new[:, 1]],
# #     y_train_new,
# #     batch_size=128,
# #     epochs=100,
# #     callbacks=[model_checkpoint, terminate_on_nan, csv_logger, early_stop],
# #     validation_data=([X_val[:, 0], X_val[:, 1]], y_val)
# # )

# # load the model
# model = load_model(f"models/{model_name}.hdf5")

# y_pred = model.predict([X_test[:, 0], X_test[:, 1]])
# y_pred = y_pred > 0.5
# print(classification_report(y_test, y_pred))